In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import xarray as xr
import seaborn as sns
import geopandas as gpd

import cartopy.feature as cf
import cartopy.crs as ccrs

from matplotlib import patches
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
dir_base = os.path.join('/','media','arturo','Arturo','Data','Brazil')

In [3]:
DATA = xr.open_dataset('/home/arturo/Downloads/IMERG_BR/IMERG_Italy_1dy_2024_01_01_2024_12_31.nc')
DATA = DATA.sel(time=slice("2024-03-01", "2024-12-31"))

lat_IM = DATA.lat.values
lon_IM = DATA.lon.values
lon2d_IM, lat2d_IM = np.meshgrid(lon_IM, lat_IM)

IMERG = DATA.PRE.values

In [4]:
obs_list = pd.read_csv(os.path.join(dir_base,'INEMET','METADATA_2024.csv'))
obs_list.head(4)

,FILE,UF,LAT,LON
0,INMET_CO_DF_A001_BRASILIA_01-01-2024_A_31-12-2...,DF,-15.789444,-47.925833
1,INMET_CO_DF_A042_BRAZLANDIA_01-01-2024_A_31-12...,DF,-15.599722,-48.131111
2,INMET_CO_DF_A045_AGUAS EMENDADAS_01-01-2024_A_...,DF,-15.596491,-47.625801
3,INMET_CO_DF_A046_GAMA (PONTE ALTA)_01-01-2024_...,DF,-15.935278,-48.137500


In [5]:
TIME_REF = pd.date_range(start=f'2024-01-01', end=f'2024-12-31', freq='d')
TIME_PD = pd.DataFrame({'datetime':TIME_REF})
print(f'Number of times: {len(TIME_PD)}')

Number of times: 366


In [6]:
for nn in range(len(obs_list)):#len(obs_list)
    obs = pd.read_csv(os.path.join(dir_base,'INEMET','1h','2024',obs_list['FILE'].values[nn]))
    obs['datetime'] = pd.to_datetime(obs['datetime'])
    # obs['datetime'] = pd.to_datetime(obs['datetime'].dt.strftime('%y%m%d %H'))
    obs = obs.set_index('datetime')
    obs_daily = obs.resample('D').sum()
    obs_daily = obs_daily.reset_index()
    obs_daily['datetime'] = obs_daily['datetime'].dt.tz_localize(None)
    
    data_rdy = pd.merge(obs_daily, TIME_PD, on='datetime', how='right')

    output = os.path.join(dir_base,'INEMET','1dy',obs_list['FILE'].values[nn])
    data_rdy.to_csv(output, header=True, index=None)